In [6]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):  # building the structure of the neural network
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

# hyperparameters
learning_rate = 1e-3
batch_size = 64  # how many data samples are run through the network before parameter update
epochs = 5  # each epoch consists of two parts: training loop, and validation/test loop

# loss function
loss_fnc = nn.CrossEntropyLoss()  # mean square error MSE, negative log likelihood NLL, LogSoftmax are other options

# Stochastic gradient descent
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [11]:
def train_loop(dataloader, model, loss_fnc, optimizer):
    size = len(dataloader.dataset)
    # setting model to training mode - import for batch normalization and dropout layers
    model.train()
    for batch, (X,y) in enumerate(dataloader):
        # compute prediction and loss
        pred = model(X)
        loss = loss_fnc(pred, y)

        # backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:  # % is modulus
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fnc):
    # set model to evaluation mode - important for batch norm and dropout layers
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # evaluating model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fnc(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {100*correct:>0.1f}%, Avg. loss: {test_loss:>8f} \n")

In [13]:
loss_fnc = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n----------------------------")
    train_loop(train_dataloader, model, loss_fnc, optimizer)
    test_loop(test_dataloader, model, loss_fnc)
    print("Done!")

Epoch 1
-----------------------
loss: 0.954131 [   64/60000]
loss: 1.001245 [ 6464/60000]
loss: 0.778219 [12864/60000]
loss: 0.965628 [19264/60000]
loss: 0.836128 [25664/60000]
loss: 0.846220 [32064/60000]
loss: 0.924189 [38464/60000]
loss: 0.879695 [44864/60000]
loss: 0.916302 [51264/60000]
loss: 0.852471 [57664/60000]
Test Error: 
 Accuracy: 67.7%, Avg. loss: 0.859725 

Done!
Epoch 2
-----------------------
loss: 0.888611 [   64/60000]
loss: 0.951626 [ 6464/60000]
loss: 0.717400 [12864/60000]
loss: 0.918215 [19264/60000]
loss: 0.796292 [25664/60000]
loss: 0.793956 [32064/60000]
loss: 0.884297 [38464/60000]
loss: 0.844903 [44864/60000]
loss: 0.874916 [51264/60000]
loss: 0.815981 [57664/60000]
Test Error: 
 Accuracy: 69.0%, Avg. loss: 0.820594 

Done!
Epoch 3
-----------------------
loss: 0.835797 [   64/60000]
loss: 0.911657 [ 6464/60000]
loss: 0.669737 [12864/60000]
loss: 0.881002 [19264/60000]
loss: 0.766001 [25664/60000]
loss: 0.753561 [32064/60000]
loss: 0.852079 [38464/60000]
los